# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
r.status_code
data_json = r.json()
rj = pd.DataFrame(data_json['items'])
rj.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,42331,Município,Tanguá,3305752,RJ,PVL02.008522/2019-58,Encaminhado à PGFN com manifestação técnica fa...,17944.101686/2020-50,2020-07-02T20:49:45Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15850000.0,1,1,15/07/2020


In [4]:
# 2) Seu código aqui
status_freq = rj.status.value_counts()
mais_freq = status_freq[:3]
mais_freq

Deferido        88
Arquivado       53
Regularizado    25
Name: status, dtype: int64

In [37]:
# 3) Seu código aqui
selecao = rj.data_status
ano_status = selecao.str.split('/' ,expand=True)
ano_status = ano_status.loc[:, 2]
ano_status

0      2020
1      2020
2      2021
3      2018
4      2019
       ... 
219    2019
220    2008
221    2009
222    2010
223    2012
Name: 2, Length: 224, dtype: object

In [49]:
# 4) Seu código aqui

ano = pd.Series(ano_status)
ano.value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2012    13
2011    10
2015     9
2009     9
2021     5
2017     5
2002     4
2006     4
2018     4
2016     4
2022     3
2004     1
2003     1
Name: 2, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [80]:
#1) Seu código aqui
# url 1 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
# url 2 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

def consulta (uf, tipo):
    if tipo == 'estado' :
        url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=xxx&tipo_interessado=Estado'
        novo_url = url.replace('xxx', uf)
        r = requests.get(novo_url)
        r.status_code
        data_json = r.json()
        df = pd.DataFrame(data_json['items'])
    else :
        url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=xxx&tipo_interessado=Munic%C3%ADpio'
        novo_url = url.replace('xxx', uf)
        r = requests.get(novo_url)
        r.status_code
        data_json = r.json()
        df = pd.DataFrame(data_json['items'])
    return df

consulta('BA', 'municipio')

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,42846,Município,Pindobaçu,2924603,BA,PVL02.001368/2020-27,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,800000.00,0,0,28/07/2020
1,39457,Município,Itatim,2916856,BA,PVL02.008123/2019-97,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2000000.00,0,1,08/07/2020
2,40308,Município,Coribe,2909109,BA,PVL02.008920/2019-74,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1845000.00,0,1,28/11/2019
3,43096,Município,Santo Antônio de Jesus,2928703,BA,PVL02.001575/2020-81,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,1191967.75,0,1,02/09/2020
4,30573,Município,Barreiras,2903201,BA,PVL02.002813/2018-51,Arquivado a pedido,17944.110173/2018-15,2019-01-10T22:48:21Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,21060000.00,0,0,17/06/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,6173,Município,Barra,2902708,BA,None,Deferido,17944.000291/2014-92,2014-05-08T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4472512.01,1,0,15/05/2014
846,6174,Município,Ribeirão do Largo,2926657,BA,None,Arquivado por decurso de prazo,17944.000292/2008-99,2008-04-14T03:00:00Z,Operação contratual interna,Caminho da Escola,Instituição Financeira Nacional,Banco do Brasil S/A,Real,359800.00,0,0,07/05/2008
847,6187,Município,Barra da Estiva,2902807,BA,None,Deferido,17944.000294/2010-01,2010-04-26T03:00:00Z,Operação contratual interna,Provias 2010,Instituição Financeira Nacional,Banco do Brasil S/A,Real,700000.00,0,0,19/05/2010
848,29853,Município,Capim Grosso,2906873,BA,PVL02.002324/2018-08,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4968418.64,0,1,12/11/2018


In [68]:
# 2) Seu código aqui
mg = consulta('MG', 'estado')
mg['status'].value_counts()
# Arquivado por decurso de prazo = 1

Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado pela STN                                        3
Arquivado                                                 3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: status, dtype: int64

In [111]:
# 3) Seu código aqui
ba = consulta('BA', 'municipio')
selecao = ba.groupby(['interessado', 'status'])[('status')].count().sort_values(ascending=False)
selecao.iloc[:1]

interessado             status  
Luís Eduardo Magalhães  Deferido    14
Name: status, dtype: int64

In [114]:
# 4) Seu código aqui 
estado_bahia = consulta('BA', 'estado')
estado_bahia.to_csv('Solicitações do estado da Bahia.csv')